In [75]:
%pylab tk

import os
import cv2
import numpy as np
from math import radians
from glob import glob
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA, RandomizedPCA
from scipy.interpolate import splprep, splev
from matplotlib import colors
import matplotlib.cm as cmx
from matplotlib.path import Path
from matplotlib.collections import LineCollection

import helpers.features as fh
import helpers.display as dh
import helpers.geometry as gh
reload(fh)
reload(dh)
reload(gh)

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_registered_outline/'

STEP_SIZE = 5 # Degrees
EXTENSION_STEP = 0.0025
WINDOW_SIZE = 0.25
NUM_SPLINE_POINTS_IN_OUTLINE = 250
NUM_SPLINE_POINTS_IN_WINDOW = 25
DEFAULT_CLASS = 'Sheep'
CLASSES = [
    {
        'name': 'Gazella',
        'identifier': 'Gaz_'
    }
]

def extract_outline_part(spline_params, degrees):
    tck, u = spline_params
    source = np.array([0.0, 0.0])
    ray = np.array(gh.pol2cart(2, radians(degrees)))
    total_space = np.linspace(0, 1, NUM_SPLINE_POINTS_IN_OUTLINE)
    
    coords = splev(total_space, tck)
    num_total_spline_points = len(coords[0])
    total_spline = np.zeros((num_total_spline_points, 2))
    total_spline[:, 0] = coords[0]
    total_spline[:, 1] = coords[1]
    
    param_for_spline = None
    for i in range(0, num_total_spline_points):
        j = i+1 if i+1 < num_total_spline_points else 0
        intersect = gh.seg_intersect(source, ray, total_spline[i, :], total_spline[j, :])
        if intersect is not None:
            dist_from_i = np.linalg.norm(intersect - total_spline[i, :])
            norm_dist_from_i = dist_from_i / np.linalg.norm(total_spline[j, :] - total_spline[i, :])
            
            if total_space[i] == 0:
                param_for_spline = total_space[j]
            if total_space[j] == 0:
                param_for_spline = total_space[i]
            else:
                param_for_spline = total_space[i] * (1-dist_from_i) + total_space[j] * dist_from_i
            break
    
    if param_for_spline is None:
        raise Exception('No intersection found')
    
    current_window_size = 0
    window_width = 0
    while window_width < WINDOW_SIZE:
        current_window_size = current_window_size + EXTENSION_STEP
        
        window_space = np.linspace(param_for_spline-current_window_size, param_for_spline+current_window_size, NUM_SPLINE_POINTS_IN_WINDOW)
        if np.any(window_space < 0):
            window_space[window_space < 0] = window_space[window_space < 0] + 1
        if np.any(window_space > 1):
            window_space[window_space > 1] = window_space[window_space > 1] - 1

        coords = splev(window_space, tck)
        window_spline = np.zeros((len(coords[0]), 2))
        window_spline[:, 0] = coords[0]
        window_spline[:, 1] = coords[1]
    
        window_width = np.cumsum(np.linalg.norm(window_spline - np.roll(window_spline, -1, axis=0), axis=1))[-2]
    return window_spline

def extract_spline(points, num_points=NUM_SPLINE_POINTS_IN_WINDOW):
    y = points[:,0].flatten()
    x = points[:,1].flatten()
    
    tck, u = splprep([y, x], s=0)
    coords = splev(np.linspace(0, 1, num_points-1), tck)
    
    spline_points = np.zeros((len(coords[0]), 2))
    spline_points[:, 0] = coords[0]
    spline_points[:, 1] = coords[1]
    
    return spline_points

def extract_spline_params(points, num_points=NUM_SPLINE_POINTS_IN_WINDOW):
    distances = np.linalg.norm(points - np.roll(points, -1), axis=1)
    distances = np.cumsum(distances / np.sum(distances))
    distances = np.append([ 0.0 ], distances)
    
    y = np.append(points[:,0].flatten(), [points[0,0]])
    x = np.append(points[:,1].flatten(), [points[0,1]])
    
    tck, u = splprep([y, x], s=0, u=distances)
    
    return tck, u

def get_class(filename):
    default = 0
    for i, cls in enumerate(CLASSES):
        if cls['identifier'] in filename:
            return i+1
    return default

def train_svc(features, classes):
    clf = LinearSVC()
    clf.fit(features, classes)
    
    return clf

def get_recall(svc, features, known_classes):
    predicted_classes = svc.predict(features)
    return float(np.count_nonzero(predicted_classes == known_classes)) / float(known_classes.shape[0])

def get_margin(svc):
    return 2 / np.linalg.norm(svc.coef_)

def map_to_display_mode(definition, closed=False):
    label, points = definition
    if closed:
        edges = np.zeros((points.shape[0], 2), dtype=np.int)
        edges[:,0] = range(0, points.shape[0])
        edges[:,1] = range(1, points.shape[0]+1)
        edges[-1,1] = 0
    else:
        edges = np.zeros((points.shape[0]-1, 2), dtype=np.int)
        edges[:,0] = range(0, points.shape[0]-1)
        edges[:,1] = range(1, points.shape[0])

    return {
        'label': label,
        'points': points,
        'edges': edges
    }
   
def feature_flatten_splines(outline_points, splines):
    return np.array([ s.flatten() for s in outline_points ])

def feature_use_pca(outline_points, splines):
    features = np.array([ s.flatten() for s in outline_points ])
    pca = PCA(n_components=4)
    reduced = pca.fit_transform(features)
    return reduced

def feature_use_distance_to_center(outline_points, splines):
    return np.array([ np.array([ np.linalg.norm(p) for p in o ]) for o in outline_points ])

def feature_use_dist_center_and_curvature(outline_points, splines):
    def curvature(x, y):
        dalpha = np.pi/1000
        xd1 = np.gradient(x)
        xd2 = np.gradient(xd1)
        yd1 = np.gradient(y)
        yd2 = np.gradient(yd1)
        return np.abs(xd1*yd2 - yd1*xd2) / np.power(xd1**2 + yd1**2, 3./2)
    
    distances = feature_use_distance_to_center(outline_points, splines)
    curvature = np.array([curvature(o[:, 0], o[:, 1]) for o in outline_points])
    
    features = np.hstack((distances, curvature))
    
    return features

def register_single_point(outlines, degrees, feature_fn=feature_use_pca, display=False):
    labels = [ o['label'] for o in outlines ]
    classes = np.array([ get_class(o['filename']) for o in outlines ])
    splines = [ extract_spline_params(o['points']) for o in outlines ]
    outline_points = [ extract_outline_part(s, degrees) for s in splines ]
    
    if display:
        dh.outlines(plt, map(map_to_display_mode, zip(labels, outline_points)))
        tmp = np.array(outline_points)
        mean_parts = [
            np.mean(tmp[classes == 0, :, :], axis=0),
            np.mean(tmp[classes == 1, :, :], axis=0)
        ]
        dh.outlines(plt, map(map_to_display_mode, zip([ "Sheep", "Gazella" ], mean_parts)))
    
    features = feature_fn(outline_points, splines)
    svc = train_svc(features, classes)
    
    recall = get_recall(svc, features, classes)
    margin = get_margin(svc)
    rm = recall*margin
    #print("Angle: {angle}, R*M: {rm}, Recall: {recall}, Margin: {margin}".format(angle=degrees, recall=recall, margin=margin, rm=rm))
    
    return {
        'angle': degrees,
        'rm': rm,
        'recall': recall,
        'margin': margin
    }

def get_averages(outlines):
    labels = set([ get_class(o['filename']) for o in outlines ])

    averages = []
    for label in labels:
        splines = np.array([
            extract_spline(o['points'], num_points=NUM_SPLINE_POINTS_IN_OUTLINE)
            for o in outlines if get_class(o['filename']) == label
        ])
        averages.append(np.mean(splines, axis=0))
        
    return zip([ "Sheep", "Gazella" ], averages)

def show_averages(outlines):
    averages = get_averages(outlines)
    dh.outlines(plt, [ map_to_display_mode(s, closed=True) for s in averages])
    
def show_averages_with_margin(outlines, results):   
    fig, axes = plt.subplots()
    plt.axis('equal')
    axes.set_ylim(-1.5, 1.5)
    axes.set_xlim(-1.5, 1.5)
    angles = [ m['angle'] for m in results ]
    current_value = 'rm'
    current_factor = 0.5
    
    margins_for_edges = []
    current_outline = np.multiply(outlines[0][1], 0.5) + np.multiply(outlines[1][1], 0.5)
    for i in range(0, current_outline.shape[0]):
            margins_for_edge = []
            j = i+1 if i+1 < current_outline.shape[0] else 0
            for mi, angle in enumerate(angles):
                source = np.array([0,0])
                ray = np.array(gh.pol2cart(2.0, radians(angle)))
                intersect = gh.seg_intersect(source, ray, current_outline[i, :], current_outline[j, :])
                if intersect is not None:
                    margins_for_edge.append(mi)
            margins_for_edges.append(margins_for_edge)
    
    def display_current_value(current_factor):
        current_values = fh.normalize(np.array([ m[current_value] for m in results ]))
        current_outline = np.multiply(outlines[0][1], current_factor) + np.multiply(outlines[1][1], (1 - current_factor))
        
        new_lines = []
        colors = []
        for i in range(0, current_outline.shape[0]):
            j = i+1 if i+1 < current_outline.shape[0] else 0
            margins_for_edge = margins_for_edges[i]                   
            margins = [ current_values[mi] for mi in margins_for_edge ]
            margin = np.mean(margins) if len(margins_for_edge) > 0 else 0
            if margin != 0:
                color = cmx.brg(margin)
            else:
                color = (1,1,1,1)
            
            new_lines.append((
                        current_outline[i, (1, 0)],
                        current_outline[j, (1, 0)]
                    ))
            colors.append(color)
        
        collection = LineCollection(new_lines, colors=colors, linewidths=2)
        while len(axes.collections) != 0:
            axes.collections[-1].remove()
        axes.add_collection(collection)
        
        fig.canvas.draw()
    
    slideraxes = fig.add_axes([0.02, 0.02, 0.96, 0.08])
    slider = Slider(slideraxes, 'Left: Gazelle, Right: Sheep', 0.0, 1.0, valinit=current_factor)
    fig._slider = slider
    def update(val):
        display_current_value(val)
    slider.on_changed(update)
    
    display_current_value(current_factor)

Populating the interactive namespace from numpy and matplotlib
Current base path: /home/stefan/Dropbox/Masterarbeit


`%matplotlib` prevents importing * from pylab and numpy


In [63]:
filenames = glob(DATA_PATH + '*.npz')

outlines = []
for f in filenames:
    basename = os.path.basename(f)
    label = ''.join([i if ord(i) < 128 else ' ' for i in basename])
    outline = np.load(f)
    outlines.append({
        'label': label,
        'filename': basename,
        'edges': outline['edges'],
        'points': outline['points']
    })

In [56]:
show_averages(outlines)

In [71]:
register_single_point(outlines, 90, display=True)

{'angle': 90,
 'margin': 2.8340970290906995,
 'recall': 0.625,
 'rm': 1.7713106431816872}

In [76]:
results = []
for angle in range(1, 361, STEP_SIZE):
    results.append(register_single_point(outlines, angle))

In [77]:
show_averages_with_margin(get_averages(outlines), results)

In [ ]:
# Spiegeln: Per Hand
# Affine Transformation ausprobieren -> estimate_transform
# Herausfinden warum es mit der Distanz nicht funktioniert
# Bessere Visualisierung für 
# Features: Abweichung vom Durchschnittsknochen